In [ ]:
import config

print(config.ALGORITHM_NAMES)
print(config.OBJECT_NAMES)

In [ ]:
import mealpy

mealpy.FFA.OriginalFFA

In [ ]:
from eval.dataset import Dataset
from view_sampler import ViewSampler
 
ds = Dataset.create_random(config.OBJ_LOCATION, 100, 1)

viewer = ViewSampler("data/mug/world.xml", camera_config=config.CAMERA_CONFIG)

ds.visualize(viewer, depth=False, annotate=False)

viewer.close()

In [ ]:
import numpy as np
import config as config
from view_sampler import ViewSampler
from algs import *
import config

from eval import eval_funcs

from eval.evaluator import Evaluator
from utils.visualize import *
from eval.eval_log import EvalLog


obj_name = "stack_rings"
alg_name = "OriginalICA"

plotter = SearchPlotter(update_freq=100, history=None, alpha=0.5)

# Create a camera configuration
world_viewer = ViewSampler(f"data/{obj_name}/world.xml", config.CAMERA_CONFIG)
sim_viewer = ViewSampler(f"data/{obj_name}/world_sim.xml", config.CAMERA_CONFIG)

alg = config.create_algorithm(alg_name, sim_viewer)

eval_func = eval_funcs.XorDiff(config.XORDIFF_PENALTY[obj_name])
evaluator = Evaluator(world_viewer, sim_viewer, eval_func=eval_func)

run_config = MealRunConfig(15)
log = EvalLog(alg)
evaluator.evaluate(alg, run_config, config.EVAL_DATASET, log=log, plot=None)

In [ ]:
world_viewer.close()
sim_viewer.close()
plotter.close()

# ANALYSIS

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
from eval.eval_log import EvalLog
from mealpy.utils.history import History
from utils.file import LogFiles
import pandas as pd

path = "grid_search/airplane"

files = LogFiles(path)

In [ ]:
data = []
for _ in files:
    try:
        data.append(files.to_dataframe(add_params=True))
    except:
        pass

df = pd.concat(data, axis=0, ignore_index=True)

In [ ]:
df.columns

In [ ]:
df.drop(columns=["alg", "sample", "ref_pos", "pred_pos", "epoch"], inplace=True)

In [ ]:
param_cols = ['empire_count', 'assimilation_coeff', 'revolution_prob',
       'revolution_rate', 'revolution_step_size']

In [ ]:
import pandas as pd

grouped = df.groupby(param_cols) 

means = grouped[["eval_loss"]].mean()
print(len(df) / len(means))

sorted_means = means.sort_values("eval_loss")
top_indices = sorted_means.index[:20]

display(sorted_means)

top_group_keys = grouped.groups.keys() & top_indices

top = []

for group_key in top_group_keys:
    corresponding_group = grouped.get_group(group_key)
    param_column = pd.Series([str(group_key)] * len(corresponding_group), index=corresponding_group.index)
    corresponding_group["params"] = param_column
    top.append(corresponding_group)

top_df = pd.concat(top, axis=0)

display(top_df)

In [ ]:
g = sns.catplot(
    data=top_df,
    x="params",
    y="eval_loss",
    kind="box",
    hue="params",
    errorbar="sd",
    showmeans=True,
    aspect=2,
)
g.ax.title.set_text(path)
g.ax.tick_params(axis="x", rotation=90)

In [ ]:
algs = pd.unique(df["revolution_rate"])
medians = {"alg": [], "median": []}
for alg_name in algs:
    mask = df["alg"] == alg_name
    # print(dataframe[mask][['eval_loss']])
    median = df[mask][["eval_loss"]].median()
    # medians[alg] = median.item()
    medians["alg"].append(alg_name)
    medians["median"].append(median.item())

In [ ]:
display(df)

In [ ]:
medians = pd.DataFrame(medians)
medians.sort_values(by="median", inplace=True)

In [ ]:
# fig, ax = plt.subplots(figsize=(10,40))
g = sns.catplot(
    data=df,
    x="alg",
    y="eval_loss",
    hue="alg",
    kind="box",
    errorbar="sd",
    aspect=8,
    order=medians.alg,
    hue_order=medians.alg,
)
g.ax.tick_params(axis="x", rotation=90)

In [ ]:
print(medians.alg.to_list())